In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import cv2
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

2025-04-15 12:14:50.056325: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 12:14:50.194676: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-15 12:14:50.385711: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-15 12:14:50.583235: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-15 12:14:50.584324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 12:14:50.906437: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
dataset_url = "http://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir=tf.keras.utils.get_file('flower_photos',origin=dataset_url, cache_dir='.',untar=True)

In [3]:
import pathlib
data_dir=pathlib.Path(data_dir)

In [4]:
flowers_images_dict={
    'roses':list(data_dir.glob('roses/*.jpg')),
    'dandelion':list(data_dir.glob('dandelion/*.jpg')),
    'daisy':list(data_dir.glob('daisy/*.jpg')),
    'sunflowers':list(data_dir.glob('sunflowers/*.jpg')),
    'tulips':list(data_dir.glob('tulips/*.jpg')),
}

In [5]:
flowers_labels_dict={
    'roses':0,
    'dandelion':1,
    'daisy':2,
    'sunflowers':3,
    'tulips':4,
}

In [6]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))
img.shape

(333, 500, 3)

In [7]:
X,Y = [],[]
for flower_name,images in flowers_images_dict.items():
    for image in images:
        img =cv2.imread(str(image))
        resized_img=cv2.resize(img,(180,180))
        X.append(resized_img)
        Y.append(flowers_labels_dict[flower_name])

In [8]:
X=np.array(X)
Y=np.array(Y)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=0)

In [10]:
X_train_scaled=X_train/255
X_test_scaled=X_test/255

In [11]:
Y_train[:10]

array([2, 0, 1, 1, 4, 0, 0, 3, 3, 4])

In [28]:
data_augmentation = Sequential([
    layers.RandomFlip("horizontal",input_shape=(180, 180, 3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
    
])

num_classes=5
model = Sequential([
    layers.Input((180, 180, 3)),
    data_augmentation,
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Add softmax
])

model.compile(loss= 'sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit(X_train_scaled, Y_train, epochs=10)

/home/iimorad31/.local/lib/python3.10/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.3398 - loss: 2.4629
Epoch 2/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.5456 - loss: 1.1209
Epoch 3/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.6119 - loss: 1.0137
Epoch 4/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.6269 - loss: 0.9703
Epoch 5/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.6513 - loss: 0.9058
Epoch 6/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 102s 1s/step - accuracy: 0.6673 - loss: 0.8927
Epoch 7/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.6904 - loss: 0.7900
Epoch 8/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.7249 - loss: 0.7601
Epoch 9/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.7046 - loss: 0.7520
Epoch 10/10
86/86 ━━━━━━━━━━━━━━━━━━━━ 154s 1s/step - accuracy: 0.7501 - loss: 0.6727


In [29]:
model.evaluate(X_test_scaled,Y_test)

29/29 ━━━━━━━━━━━━━━━━━━━━ 8s 265ms/step - accuracy: 0.7147 - loss: 0.7324


[0.7805808782577515, 0.7113289833068848]